In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('./data/debtors info.xlsx').drop_duplicates(subset=['dossier_nr']).set_index('dossier_nr')
df

In [ ]:
for col in ['datum_betekening', 'datum_afsluiten', 'datum_toevoegen', 'datum_laatste_betaling']:
    df[col] = pd.to_datetime(df[col])

# Work through columnns

In [ ]:
df.columns

### Case duration (for landmarking)

### Case type

In [ ]:
case_types = pd.get_dummies(df.soort_zaak_code, prefix='case_type')
case_types

### Has previously paid

In [ ]:
has_previously_paid = df['datum_laatste_betaling'].apply(lambda x: 0 if pd.isnull(x) else 1)
has_previously_paid.name = 'has_previously_paid'
has_previously_paid = pd.DataFrame(has_previously_paid)
has_previously_paid

### Debt amount

In [ ]:
debt_amount = pd.DataFrame(df['bedrag_hoofdsom'])
debt_amount.columns = ['debt_amount']
debt_amount['debt_amount'] = debt_amount.debt_amount.apply(lambda x: x if x < 2000 else 2000)
debt_amount

# Load debtor_info

In [ ]:
debtor_info = pd.read_excel('./data/debtors info.xlsx', sheet_name = 'debiteuren').drop_duplicates(subset=['dossier_nr']).set_index('dossier_nr')
debtor_info

In [ ]:
debtor_info = debtor_info[debtor_info.debiteur_RP_JN == 0]

In [ ]:
debtor_info.describe()

### Debtor age

In [ ]:
debtor_age = pd.DataFrame(debtor_info['debiteur_leeftijd'])
debtor_age.columns = ['debtor_age']
debtor_age['debtor_age'] = debtor_age['debtor_age'].apply( lambda x: int(x) if x != 1000 else (debtor_age.debtor_age.mean())).astype(int)
debtor_age

###  Debtor Sex

In [ ]:
debtor_sex = pd.DataFrame(debtor_info['debiteur_geslacht'].apply(lambda x: x if x != 0 else 1))
debtor_sex.columns = ['debtor_sex']
debtor_sex

### Debtor marital status

In [ ]:
debtor_marital_status = pd.DataFrame(debtor_info['debiteur_burgerlijke_staat'].apply(lambda x: x if x < 4 else 'other'))
debtor_marital_status = pd.get_dummies(debtor_marital_status.debiteur_burgerlijke_staat, prefix='marital_status')
debtor_marital_status

## Boolean columns

In [ ]:
boolean_column_names = ['debiteur_adres_io', 'debiteur_adres_vow', 'debiteur_telefoonnummer_JN', 'debiteur_email_JN', 
                   'debiteur_bankrekening_JN', 'debiteur_NP_bedrijf_JN', 'debiteur_BSN_bekend_JN', 'debiteur_werkgever_bekend_JN',
                  'debiteur_overleden_JN', 'debiteur_agressief_JN', 'debiteur_detentie_JN', 'debiteur_bewind_JN', 'debiteur_insolvent_JN', 'debiteur_curatele_JN',
                  'debiteur_SHV_gemeente_JN']
boolean_columns = debtor_info[boolean_column_names]
boolean_columns

## Postal code

In [ ]:
debtor_info.debiteur_postcode_4_cijfers.value_counts(dropna=False)

In [ ]:
debtor_info.debiteur_postcode_4_cijfers = debtor_info.debiteur_postcode_4_cijfers.fillna(debtor_info.debiteur_postcode_4_cijfers.mode(0)[0])

In [ ]:
postal_code_income = pd.read_excel('./data/Besteedbaar-inkomen-postcode-2004-2014.xlsx',  index_col = 0)
postal_code_income['Disposable Income'] = postal_code_income['Disposable Income'].replace({'.': 24.7})
postal_code_income

In [ ]:
def get_disposable_income(postcal_code):
    try: 
        return postal_code_income.loc[postcal_code, 'Disposable Income']
    except KeyError:
        return 24.7

In [ ]:
disposable_income = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_disposable_income(x)))
disposable_income.columns = ['disposable_income']

In [ ]:
postal_code_info = pd.read_excel('./data/cbs_pc4_2020_v1.xlsx', skiprows=1, index_col=0)
postal_code_info = postal_code_info.apply(lambda x: np.where(x < 0, np.nan,x))
postal_code_info

In [ ]:
def get_fraction_under_25(postal_code):
    return (postal_code_info.loc[postal_code, 'INW_014'] + postal_code_info.loc[postal_code, 'INW_1524']) / postal_code_info.loc[postal_code, 'INWONER']

def get_fraction_under_45(postal_code):
    return (postal_code_info.loc[postal_code, 'INW_014'] + postal_code_info.loc[postal_code, 'INW_1524']  + postal_code_info.loc[postal_code, 'INW_2544']) / postal_code_info.loc[postal_code, 'INWONER']

In [ ]:
fraction_under_25 = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_fraction_under_25(x)))
fraction_under_25.columns = ['fraction_under_25']
fraction_under_45 = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_fraction_under_45(x)))

In [ ]:
def get_fraction_rented_adresses(postal_code):
    return postal_code_info.loc[postal_code, 'P_HUURWON'] / 100
fraction_rented_houses = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_fraction_rented_adresses(x)))
fraction_rented_houses.columns = ['fraction_rented_houses']

In [ ]:
def get_average_house_value(postal_code):
    return postal_code_info.loc[postal_code, 'WOZWONING']
average_house_value = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_average_house_value(x)))
average_house_value.columns = ['average_house_value']

In [ ]:
def get_average_adresses_per_square_kilometer(postal_code):
    return postal_code_info.loc[postal_code, 'OAD']
average_adresses_per_square_kilometer = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_average_adresses_per_square_kilometer(x)))
average_adresses_per_square_kilometer.columns = ['average_adresses_per_square_kilometer']

In [ ]:
def get_percentage_unemployed(postal_code):
    return postal_code_info.loc[postal_code, 'UITKMINAOW'] /  postal_code_info.loc[postal_code, 'INWONER']
percentage_unemployed = pd.DataFrame(debtor_info.debiteur_postcode_4_cijfers.apply(lambda x: get_percentage_unemployed(x)))
percentage_unemployed.columns = ['percentage_unemployed']

# Merging all variables together

In [ ]:
# fraction_rented_houses, average_house_value, average_adresses_per_square_kilometer, percentage_unemployed

In [ ]:
variables = [df['debiteur_relatie_nr'], case_types, debt_amount, debtor_age, debtor_sex, debtor_marital_status, boolean_columns, disposable_income, fraction_under_25, ]

In [ ]:
result = pd.concat(variables, axis=1, join="inner")

In [ ]:
result

In [ ]:
result.iloc[:, 1:]

In [ ]:
result.iloc[:1000, 1:].info()

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(result.reset_index().iloc[:500, 2:], title="Pandas Profiling Report", explorative=True)
profile.to_file("your_report.html")

In [ ]:
result.reset_index().iloc[:500, 2:]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.heatmap(result.reset_index().iloc[:, :].corr());

In [ ]:
result.to_csv('./data/dataset.csv')